# Day 2：データ選択・集約のための基礎的なSQL操作
大規模なデータの管理およびそれに基づくデータ分析を行っている企業や研究室では，何かしらのデータベースマネジメントシステム（DBMS）をデータを処理している．今日，データベースは
* [関係データベース](https://ja.wikipedia.org/wiki/関係データベース)
* [NoSQL](https://ja.wikipedia.org/wiki/NoSQL)（RDB以外のデータベース）

に大別される．このうち，関係データベースは長い歴史があり，構造化されたデータを保存・管理・検索・集約するために現在でも広く使われている．

Day 2からDay 5では，関係データベースの管理システム（DBMS）の中でも，簡単に環境が構築できる軽量なシステムである[SQLite](https://www.sqlite.org/)を用いる．SQLiteの関係データベース上に保存された小売店の購買データにターゲットに，Excelでは扱いきれないサイズのデータの前処理・分析を体験する．


## SQL利用環境セットアップ
SQLiteはWindows/Mac/LinuxどのOSでもインストールして利用することができる．本演習は環境構築の手間をできるだけ減らすために，Google Colaboratory上でSQLiteを用いる．

Googleにログインした状態で[本演習サイト](https://data-analytics2022.hontolab.org)の各回の「ドリル（模範解答なし）」のリンクをクリックすると，あなた用のGoogle Colaboratoyがブラウザ上で立ち上がり，各回のドリル・課題を行うためのページが表示されたら．ページが表示されたら，以下のコードが書かれた箇所（セル）を実行しよう．すると，Google Colaboratory上でSQLite（ipython-sql）の実行環境が用意される．

In [1]:
# ipython-sqlがインストールされていない場合
!pip install ipython-sql

You should consider upgrading via the '/Users/yamamoto/.anyenv/envs/pyenv/versions/3.9.8/bin/python3.9 -m pip install --upgrade pip' command.


英文が数行表示された後，SQLiteがインストールされる．
次に演習で用いる購買データが収められたSQLiteデータベースをダウンロードする．
以下のコードを実行すると，Google Colaboratory上にデータベースがダウンロードされる．

In [ ]:
# SQLiteデータベース（data-analytics-lecture.db）のファイルをダウンロード
!wget https://github.com/hontolab-courses/data-analytics-2022/raw/main/data/data-analytics-lecture.db

データベースがダウンロードできたら，データをロードしておこう．
以下のコードを実行すると，Google Colaboratory上でダウンロードしたSQLiteデータベースを操作できるようになる．

In [2]:
%load_ext sql
%sql sqlite:///../data/data-analytics-lecture.db

なお，Google Colaboratoryはセッションを閉じると，環境が初期化され，以前に自前でインストールしたライブラリが削除される．
Google Colaboratoryを使って演習課題に取り組む際には，上記コードを毎回忘れず実行すること．

---

## Jupyter上でのSQL実行方法

Google Colaboratoryでは，以下のようにセルの冒頭に`%%sql`と書き，2行目以降にSQL文を書くとSQLを実行することができる．
SQL文に`LIMIT`を付けないと，テーブル内の全データが表示されてしまうので注意．

In [4]:
%%sql

SELECT * FROM receipt LIMIT 10; 

 * sqlite:///../data/data-analytics-lecture.db
Done.


sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20170101,1483228800,S12013,1112,1,ZZ000000000000,P090102011,1,180
20170101,1483228800,S12013,1142,2,ZZ000000000000,P070901024,1,198
20170101,1483228800,S12013,1172,2,ZZ000000000000,P070202007,1,138
20170101,1483228800,S12014,1152,1,CS014411000048,P080401010,1,110
20170101,1483228800,S12014,1152,2,CS014411000048,P070708081,1,338
20170101,1483228800,S12014,1172,1,ZZ000000000000,P071401022,1,2400
20170101,1483228800,S12029,112,1,CS029214000004,P060203001,1,98
20170101,1483228800,S12029,112,2,CS029214000004,P060701001,1,98
20170101,1483228800,S12029,1132,1,ZZ000000000000,P060104003,1,80
20170101,1483228800,S12029,1192,1,CS029414000005,P050102001,1,60


---
## Day 2 の課題

これまでに何名の顧客が何日ショッピングを行ったのかを分析したい．
レシート明細テーブル（`receipt`）を用いて顧客の購買頻度を分析し，以下の項目について分析結果を表示せよ：
* 購買頻度（これまでに店舗を利用した日数）
* 購買頻度に対応する顧客の数
* 該当する購買頻度以下の顧客数の累積値

ただし，顧客ID（`customer_id`）が"Z"から始まるのものは非会員を表すため，除外して分析すること．

In [7]:
%%sql

WITH action_count_table AS (
    SELECT
        customer_id,
        COUNT(DISTINCT sales_ymd) AS action_day_count
    FROM
        receipt
    WHERE
        customer_id NOT LIKE "Z%"
    GROUP BY
        customer_id
)
SELECT
    action_day_count,
    COUNT(*) AS customer_num,
    SUM(COUNT(*)) OVER (ORDER BY action_day_count) AS cumulative_customer_num
FROM
    action_count_table
GROUP BY
    action_day_count; 

 * sqlite:///../data/data-analytics-lecture.db
Done.


action_day_count,customer_num,cumulative_customer_num
1,2761,2761
2,1499,4260
3,881,5141
4,598,5739
5,424,6163
6,384,6547
7,369,6916
8,322,7238
9,272,7510
10,234,7744


---
## ノック

### Knock 11: 選択
レシート明細のテーブル（`receipt`）から全項目の先頭10件を表示し，どのようなレコードが保存されているか確認せよ．

In [8]:
%%sql

SELECT * FROM receipt LIMIT 10; 

 * sqlite:///../data/data-analytics-lecture.db
Done.


sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20170101,1483228800,S12013,1112,1,ZZ000000000000,P090102011,1,180
20170101,1483228800,S12013,1142,2,ZZ000000000000,P070901024,1,198
20170101,1483228800,S12013,1172,2,ZZ000000000000,P070202007,1,138
20170101,1483228800,S12014,1152,1,CS014411000048,P080401010,1,110
20170101,1483228800,S12014,1152,2,CS014411000048,P070708081,1,338
20170101,1483228800,S12014,1172,1,ZZ000000000000,P071401022,1,2400
20170101,1483228800,S12029,112,1,CS029214000004,P060203001,1,98
20170101,1483228800,S12029,112,2,CS029214000004,P060701001,1,98
20170101,1483228800,S12029,1132,1,ZZ000000000000,P060104003,1,80
20170101,1483228800,S12029,1192,1,CS029414000005,P050102001,1,60


### Knock 12: 射影1
レシート明細のテーブル（`receipt`）から売上日（`sales_ymd`），顧客ID（`customer_id`），商品コード（`product_cd`），売上金額（`amount`）の順に列を指定し，10件レコードを表示せよ．

In [9]:
%%sql

SELECT sales_ymd, customer_id, product_cd, amount FROM receipt LIMIT 10; 

 * sqlite:///../data/data-analytics-lecture.db
Done.


sales_ymd,customer_id,product_cd,amount
20170101,ZZ000000000000,P090102011,180
20170101,ZZ000000000000,P070901024,198
20170101,ZZ000000000000,P070202007,138
20170101,CS014411000048,P080401010,110
20170101,CS014411000048,P070708081,338
20170101,ZZ000000000000,P071401022,2400
20170101,CS029214000004,P060203001,98
20170101,CS029214000004,P060701001,98
20170101,ZZ000000000000,P060104003,80
20170101,CS029414000005,P050102001,60


### Knock 13: 射影2
レシート明細のテーブル（`receipt`）から売上日（`sales_ymd`），顧客ID（`customer_id`），商品コード（`product_cd`），売上金額（`amount`）の順に列を指定し，10件レコードを表示せよ．ただし，列`sales_ymd`は列名を`sales_date`に変更して表示すること．

In [10]:
%%sql

SELECT 
    sales_ymd AS sales_date,
    customer_id,
    product_cd,
    amount
FROM
    receipt
LIMIT 10; 

 * sqlite:///../data/data-analytics-lecture.db
Done.


sales_date,customer_id,product_cd,amount
20170101,ZZ000000000000,P090102011,180
20170101,ZZ000000000000,P070901024,198
20170101,ZZ000000000000,P070202007,138
20170101,CS014411000048,P080401010,110
20170101,CS014411000048,P070708081,338
20170101,ZZ000000000000,P071401022,2400
20170101,CS029214000004,P060203001,98
20170101,CS029214000004,P060701001,98
20170101,ZZ000000000000,P060104003,80
20170101,CS029414000005,P050102001,60


### Knock 14: 選択2
レシート明細のテーブル（`receipt`）から売上日（`sales_ymd`），顧客ID（`customer_id`），商品コード（`product_cd`），売上金額（`amount`）の順に列を指定し，以下の条件を満たすレコードをすべて抽出せよ．

* 顧客IDが"CS018205000001"

In [11]:
%%sql

SELECT 
    sales_ymd,
    customer_id,
    product_cd,
    amount
FROM
    receipt
WHERE
    customer_id = "CS018205000001"

 * sqlite:///../data/data-analytics-lecture.db
Done.


sales_ymd,customer_id,product_cd,amount
20170614,CS018205000001,P060702015,108
20170614,CS018205000001,P050206001,990
20180414,CS018205000001,P071101002,278
20180414,CS018205000001,P060104007,600
20180911,CS018205000001,P071401005,1100
20180911,CS018205000001,P071401012,2200
20190216,CS018205000001,P040101002,218
20190216,CS018205000001,P071005024,102
20190226,CS018205000001,P071401020,2200
20190226,CS018205000001,P070902035,168


### Knock 15: 選択3
レシート明細のテーブル（`receipt`）から売上日（`sales_ymd`），顧客ID（`customer_id`），商品コード（`product_cd`），売上金額（`amount`）の順に列を指定し，以下の条件を満たすレコードをすべて抽出せよ．

* 顧客IDが"CS018205000001"
* 売上金額が1000以上

In [12]:
%%sql

SELECT 
    sales_ymd,
    customer_id,
    product_cd,
    amount
FROM
    receipt
WHERE
    customer_id = "CS018205000001" AND
    amount >= 1000;

 * sqlite:///../data/data-analytics-lecture.db
Done.


sales_ymd,customer_id,product_cd,amount
20180911,CS018205000001,P071401005,1100
20180911,CS018205000001,P071401012,2200
20190226,CS018205000001,P071401020,2200


### Knock 16: 選択4
レシート明細のテーブル（`receipt`）から売上日（`sales_ymd`），顧客ID（`customer_id`），商品コード（`product_cd`），売上数量（`quantity`），売上金額（`amount`）の順に列を指定し，以下の条件を満たすレコードをすべて抽出せよ．

* 顧客IDが"CS018205000001"
* 売上金額が1000以上または売上数量が5以上

In [13]:
%%sql

SELECT 
    sales_ymd,
    customer_id,
    product_cd,
    quantity,
    amount
FROM
    receipt
WHERE
    customer_id = "CS018205000001" AND
    (amount >= 1000 OR quantity >= 5);

 * sqlite:///../data/data-analytics-lecture.db
Done.


sales_ymd,customer_id,product_cd,quantity,amount
20170614,CS018205000001,P050206001,5,990
20180414,CS018205000001,P060104007,6,600
20180911,CS018205000001,P071401005,1,1100
20180911,CS018205000001,P071401012,1,2200
20190226,CS018205000001,P071401020,1,2200


### Knock 17: 選択5
レシート明細のテーブル（`receipt`）から売上日（`sales_ymd`），顧客ID（`customer_id`），商品コード（`product_cd`），売上金額（`amount`）の順に列を指定し，以下の条件を満たすレコードをすべて抽出せよ．

* 顧客IDが"CS018205000001"
* 売上金額が1000以上2000以下

In [14]:
%%sql

SELECT 
    sales_ymd,
    customer_id,
    product_cd,
    amount
FROM
    receipt
WHERE
    customer_id = "CS018205000001" AND
    amount BETWEEN 1000 AND 2000;

 * sqlite:///../data/data-analytics-lecture.db
Done.


sales_ymd,customer_id,product_cd,amount
20180911,CS018205000001,P071401005,1100


### Knock 18: 選択6
レシート明細のテーブル（`receipt`）から売上日（`sales_ymd`），顧客ID（`customer_id`），商品コード（`product_cd`），売上金額（`amount`）の順に列を指定し，以下の条件を満たすレコードをすべて抽出せよ．

* 顧客IDが"CS018205000001"
* 商品コードが"P071401019"以外

In [15]:
%%sql

SELECT 
    sales_ymd,
    customer_id,
    product_cd,
    amount
FROM
    receipt
WHERE
    customer_id = "CS018205000001" AND
    product_cd != "P071401019";

 * sqlite:///../data/data-analytics-lecture.db
Done.


sales_ymd,customer_id,product_cd,amount
20170614,CS018205000001,P060702015,108
20170614,CS018205000001,P050206001,990
20180414,CS018205000001,P071101002,278
20180414,CS018205000001,P060104007,600
20180911,CS018205000001,P071401005,1100
20180911,CS018205000001,P071401012,2200
20190216,CS018205000001,P040101002,218
20190216,CS018205000001,P071005024,102
20190226,CS018205000001,P071401020,2200
20190226,CS018205000001,P070902035,168


### Knock 19: 選択7
下記SQL文について，問い合わせ結果が変わらないよう`OR`を`AND`に書き換えよ．

```
SELECT 
    *
FROM
    store
WHERE
    NOT (prefecture_cd == 13 OR floor_area > 900);
```

In [16]:
%%sql

SELECT 
    *
FROM
    store
WHERE
    prefecture_cd != 13 AND
    floor_area <= 900;

 * sqlite:///../data/data-analytics-lecture.db
Done.


store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.022,35.66122,808
S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890
S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831


### Knock 20: LIKE句1
店舗テーブル（`store`）から，店舗コード（`store_cd`）が"S14"で始まるものだけを全項目抽出し，10件だけ表示せよ．

In [17]:
%%sql

SELECT 
    *
FROM
    store
WHERE
    store_cd LIKE "S14%"
LIMIT 10;

 * sqlite:///../data/data-analytics-lecture.db
Done.


store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S14006,葛が谷店,14,神奈川県,神奈川県横浜市都筑区葛が谷,カナガワケンヨコハマシツヅキククズガヤ,045-123-4031,139.5633,35.53573,1886
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890
S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341
S14021,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962
S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838
S14023,川崎店,14,神奈川県,神奈川県川崎市川崎区本町二丁目,カナガワケンカワサキシカワサキクホンチョウニチョウメ,044-123-4037,139.7028,35.53599,1804
S14024,三田店,14,神奈川県,神奈川県川崎市多摩区三田四丁目,カナガワケンカワサキシタマクミタヨンチョウメ,044-123-4038,139.5424,35.6077,972
S14025,大和店,14,神奈川県,神奈川県大和市下和田,カナガワケンヤマトシシモワダ,046-123-4039,139.468,35.43414,1011
S14026,辻堂西海岸店,14,神奈川県,神奈川県藤沢市辻堂西海岸二丁目,カナガワケンフジサワシツジドウニシカイガンニチョウメ,046-123-4040,139.4466,35.32464,1732


### Knock 21: LIKE句2
店舗テーブル（`store`）から横浜市の店舗だけを全項目表示せよ．

In [18]:
%%sql

SELECT 
    *
FROM
    store
WHERE
    address LIKE "%横浜市%";

 * sqlite:///../data/data-analytics-lecture.db
Done.


store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S14006,葛が谷店,14,神奈川県,神奈川県横浜市都筑区葛が谷,カナガワケンヨコハマシツヅキククズガヤ,045-123-4031,139.5633,35.53573,1886
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890
S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341
S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495
S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548
S14042,新山下店,14,神奈川県,神奈川県横浜市中区新山下二丁目,カナガワケンヨコハマシナカクシンヤマシタニチョウメ,045-123-4047,139.6593,35.43894,1044
S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831
S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657


### Knock 22: 正規表現1
顧客テーブル（`customer`）から，ステータスコード（`status_cd`）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し，10件だけ表示せよ．

In [19]:
%%sql

SELECT 
    *
FROM
    customer
WHERE
    status_cd REGEXP "^[A-F].+"
LIMIT 10

 * sqlite:///../data/data-analytics-lecture.db
Done.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS001115000010,藤沢 涼,1,女性,2006-05-16 00:00:00,12,144-0056,東京都大田区西六郷**********,S13001,20150417,B-20100405-9
CS001205000006,福士 明,0,男性,1993-06-12 00:00:00,25,144-0056,東京都大田区西六郷**********,S13001,20161111,B-20100224-C
CS001214000009,平尾 奈月,1,女性,1990-05-02 00:00:00,28,144-0056,東京都大田区西六郷**********,S13001,20160108,D-20100902-D
CS001214000017,小野 寿々花,1,女性,1996-06-05 00:00:00,22,144-0056,東京都大田区西六郷**********,S13001,20150409,C-20101006-C
CS001214000048,牧 エリカ,1,女性,1991-06-20 00:00:00,27,144-0056,東京都大田区西六郷**********,S13001,20160830,A-20100929-C
CS001214000052,宮脇 恵梨香,1,女性,1993-12-01 00:00:00,25,144-0055,東京都大田区仲六郷**********,S13001,20161008,E-20100617-C
CS001215000040,大原 奈央,1,女性,1995-07-10 00:00:00,23,144-0055,東京都大田区仲六郷**********,S13001,20151211,C-20081022-B
CS001215000097,竹中 あさみ,1,女性,1990-07-25 00:00:00,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
CS001304000006,塩谷 竜也,0,男性,1980-02-24 00:00:00,39,144-0055,東京都大田区仲六郷**********,S13001,20151012,C-20080424-7
CS001305000005,前島 雅彦,0,男性,1979-01-02 00:00:00,40,144-0045,東京都大田区南六郷**********,S13001,20150519,C-20100420-D


### Knock 23: 正規表現2
顧客テーブル（`customer`）から，ステータスコード（`status_cd`）の末尾が数字の1〜9で始まるデータを全項目抽出し，10件だけ表示せよ．

In [20]:
%%sql

SELECT 
    *
FROM
    customer
WHERE
    status_cd REGEXP ".+[1-9]$"
LIMIT 10;

 * sqlite:///../data/data-analytics-lecture.db
Done.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS001113000004,葛西 莉央,1,女性,2003-02-22 00:00:00,16,144-0056,東京都大田区西六郷**********,S13001,20151105,7-20100308-1
CS001114000005,安 里穂,1,女性,2004-11-22 00:00:00,14,144-0056,東京都大田区西六郷**********,S13001,20160412,5-20100731-7
CS001115000010,藤沢 涼,1,女性,2006-05-16 00:00:00,12,144-0056,東京都大田区西六郷**********,S13001,20150417,B-20100405-9
CS001211000025,河野 夏希,1,女性,1996-06-09 00:00:00,22,140-0013,東京都品川区南大井**********,S13001,20160505,4-20100322-2
CS001212000027,杉山 なぎさ,1,女性,1991-03-25 00:00:00,28,210-0022,神奈川県川崎市川崎区池田**********,S13001,20150731,3-20080127-2
CS001212000031,平塚 恵望子,1,女性,1990-07-26 00:00:00,28,210-0007,神奈川県川崎市川崎区駅前本町**********,S13001,20150822,2-20090906-2
CS001212000046,伊東 愛,1,女性,1994-09-08 00:00:00,24,210-0014,神奈川県川崎市川崎区貝塚**********,S13001,20160615,1-20080811-2
CS001212000070,浜本 愛,1,女性,1996-01-14 00:00:00,23,210-0831,神奈川県川崎市川崎区観音**********,S13001,20160918,4-20101018-2
CS001212000098,菊地 りえ,9,不明,1995-01-19 00:00:00,24,210-0842,神奈川県川崎市川崎区渡田東町**********,S13001,20170604,3-20090405-2
CS001213000018,市川 瞳,1,女性,1996-01-16 00:00:00,23,144-0055,東京都大田区仲六郷**********,S13001,20150721,1-20080519-2


### Knock 24: 正規表現3
顧客テーブル（`customer`）から，ステータスコード（`status_cd`）の先頭がアルファベットのA〜Fで始まり，末尾が数字の1〜9で始まるデータを全項目抽出し，10件だけ表示せよ．

In [21]:
%%sql

SELECT 
    *
FROM
    customer
WHERE
    status_cd REGEXP "^[A-F].+[1-9]$"
LIMIT 10;

 * sqlite:///../data/data-analytics-lecture.db
Done.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS001115000010,藤沢 涼,1,女性,2006-05-16 00:00:00,12,144-0056,東京都大田区西六郷**********,S13001,20150417,B-20100405-9
CS001215000097,竹中 あさみ,1,女性,1990-07-25 00:00:00,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
CS001304000006,塩谷 竜也,0,男性,1980-02-24 00:00:00,39,144-0055,東京都大田区仲六郷**********,S13001,20151012,C-20080424-7
CS001311000059,浜口 菜々美,1,女性,1985-04-22 00:00:00,33,212-0004,神奈川県川崎市幸区小向西町**********,S13001,20160303,A-20090211-2
CS001314000060,若杉 憲史,9,不明,1980-12-26 00:00:00,38,144-0055,東京都大田区仲六郷**********,S13001,20150625,A-20090214-2
CS001315000074,上村 沙知絵,1,女性,1984-12-29 00:00:00,34,144-0035,東京都大田区南蒲田**********,S13001,20150920,A-20080524-1
CS001315000180,長浜 瞳,1,女性,1987-03-19 00:00:00,32,144-0052,東京都大田区蒲田**********,S13001,20151122,B-20081018-1
CS001411000027,長瀬 礼子,1,女性,1969-05-15 00:00:00,49,210-0024,神奈川県川崎市川崎区日進町**********,S13001,20150301,A-20091001-1
CS001411000037,矢沢 路子,1,女性,1974-11-26 00:00:00,44,210-0866,神奈川県川崎市川崎区水江町**********,S13001,20150604,A-20090308-1
CS001411000054,筒井 花,1,女性,1973-06-19 00:00:00,45,210-0007,神奈川県川崎市川崎区駅前本町**********,S13001,20150705,A-20091019-8


### Knock 25: ソート1
顧客テーブル（`customer`）のレコードを高年齢順にソートし，先頭10レコードを前項目表示せよ．

In [22]:
%%sql

SELECT 
    *
FROM
    customer
ORDER BY
    age DESC
LIMIT 10;

 * sqlite:///../data/data-analytics-lecture.db
Done.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS027803000004,内村 拓郎,0,男性,1929-01-12 00:00:00,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
CS026813000004,吉村 朝陽,1,女性,1928-12-14 00:00:00,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
CS018811000003,熊沢 美里,1,女性,1929-01-07 00:00:00,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
CS016815000002,山元 美紀,1,女性,1929-02-22 00:00:00,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
CS013801000003,天野 拓郎,0,男性,1929-01-15 00:00:00,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
CS003813000014,村山 菜々美,1,女性,1928-11-26 00:00:00,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
CS001814000022,鶴田 里穂,1,女性,1929-01-28 00:00:00,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
CS039802000003,向井 ヒロ,0,男性,1929-06-08 00:00:00,89,167-0051,東京都杉並区荻窪**********,S13039,20151019,0-00000000-0
CS038815000019,田辺 怜奈,1,女性,1930-01-29 00:00:00,89,134-0084,東京都江戸川区東葛西**********,S13038,20150829,0-00000000-0
CS038814000001,吉川 郁恵,1,女性,1929-09-13 00:00:00,89,279-0042,千葉県浦安市東野**********,S13038,20150708,C-20090908-D


### Knock 26: ソート2
顧客テーブル（`customer`）のレコードを若い順（同じ年齢の場合は誕生日が遅い順）にソートし，先頭10レコードを前項目表示せよ．

In [23]:
%%sql

SELECT 
    *
FROM
    customer
ORDER BY
    age ASC, birth_day DESC
LIMIT 10;

 * sqlite:///../data/data-analytics-lecture.db
Done.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS035114000004,大村 美里,1,女性,2007-11-25 00:00:00,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
CS022103000002,福山 はじめ,9,不明,2007-10-02 00:00:00,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
CS002113000009,柴田 真悠子,1,女性,2007-09-17 00:00:00,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
CS004115000014,松井 京子,1,女性,2007-08-09 00:00:00,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
CS002114000010,山内 遥,1,女性,2007-06-03 00:00:00,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
CS025115000002,小柳 夏希,1,女性,2007-04-18 00:00:00,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
CS002113000025,広末 まなみ,1,女性,2007-03-30 00:00:00,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
CS033112000003,長野 美紀,1,女性,2007-03-22 00:00:00,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
CS007115000006,福岡 瞬,1,女性,2007-03-10 00:00:00,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F
CS014113000008,矢口 莉緒,1,女性,2007-03-05 00:00:00,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6


### Knock 27: 集計1
レシート明細テーブル（`receipt`）のレコード数を求めよ．

In [24]:
%%sql

SELECT COUNT(*) FROM receipt;

 * sqlite:///../data/data-analytics-lecture.db
Done.


COUNT(*)
104681


### Knock 28: 集計2
レシート明細テーブル（`receipt`）中の顧客ID（`customer_id`）の総数（重複を除く）を求めよ．

In [25]:
%%sql

SELECT COUNT(DISTINCT customer_id) FROM receipt;

 * sqlite:///../data/data-analytics-lecture.db
Done.


COUNT(DISTINCT customer_id)
8307


### Knock 29: 集約演算1
レシート明細テーブル（`df_receipt`）に対し，店舗コード（`store_cd`）ごとに売上金額の合計（`total_amount`）と売上数量の合計（`total_quantity`）を計算し表示せよ．なお，表示件数は先頭の10件でよい．



In [26]:
%%sql

SELECT
    store_cd,
    SUM(amount) AS total_amount,
    SUM(quantity) AS total_quantity
FROM
    receipt
GROUP BY
    store_cd
LIMIT 10;

 * sqlite:///../data/data-analytics-lecture.db
Done.


store_cd,total_amount,total_quantity
S12007,638761,2099
S12013,787513,2425
S12014,725167,2358
S12029,794741,2555
S12030,684402,2403
S13001,811936,2347
S13002,727821,2340
S13003,764294,2197
S13004,779373,2390
S13005,629876,2004


### Knock 30: 集約演算2
レシート明細テーブル（`df_receipt`）に対し，顧客ID（`customer_id`）ごとに最も新しい売上日と最も古い売り上げ日を求め，それらが異なる顧客IDを10件表示せよ．なお，表示項目は顧客IDと最も新しい売り上げ日（`latest_sales_ymd`），最も古い売り上げ日（`oldest_sales_ymd`）とする．

In [27]:
%%sql

SELECT
    customer_id,
    MAX(sales_ymd) AS latest_sales_ymd,
    MIN(sales_ymd) AS oldest_sales_ymd
FROM
    receipt
GROUP BY
    customer_id
/* HAVING句はGROUP BYで集約した結果に対して条件付けをできる */
HAVING
    MAX(sales_ymd) != MIN(sales_ymd)   
LIMIT 10;

 * sqlite:///../data/data-analytics-lecture.db
Done.


customer_id,latest_sales_ymd,oldest_sales_ymd
CS001114000005,20190731,20180503
CS001115000010,20190405,20171228
CS001205000004,20190625,20170914
CS001205000006,20190224,20180207
CS001214000009,20190902,20170306
CS001214000017,20191006,20180828
CS001214000048,20190929,20171109
CS001214000052,20190617,20180208
CS001215000005,20181021,20170206
CS001215000040,20171022,20170214


### Knock 31: 集約演算3
レシート明細テーブル（`df_receipt`）に対し，店舗コード（`store_cd`）ごとに売上金額（`amount`）の平均を計算し，平均売り上げ金額の上位5件の店舗を表示せよ．

In [28]:
%%sql

SELECT
    store_cd,
    AVG(amount) AS avg_amount
FROM
    receipt
GROUP BY
    store_cd
ORDER BY
    AVG(amount) DESC
LIMIT 5;

 * sqlite:///../data/data-analytics-lecture.db
Done.


store_cd,avg_amount
S13052,402.86746987951807
S13015,351.11196043165467
S13003,350.9155188246097
S14010,348.79126213592235
S13001,348.4703862660944


### Knock 32: 集約演算4
レシート明細テーブル（`df_receipt`）に対し，店舗コード（`store_cd`）ごとに売上金額（`amount`）の総額を計算し，売り上げ総額が800,000以上の店舗を表示せよ．

In [29]:
%%sql

SELECT
    store_cd,
    SUM(amount) AS total_amount
FROM
    receipt
GROUP BY
    store_cd
HAVING
    SUM(amount) >= 800000;

 * sqlite:///../data/data-analytics-lecture.db
Done.


store_cd,total_amount
S13001,811936
S13008,809288
S13009,808870
S13019,827833
S14011,805724
S14026,824537


### Knock 33: Window関数1
レシート明細テーブル（`receipt`）の各レコードに対して売上金額（`amount`）が高い順にランク（順位）を付与し，先頭10件を抽出せよ．その際，項目として顧客ID（`customer_id`），売上金額（`amount`），付与したランクを表示させること．なお，売上金額が等しい場合は，同一順位を付与するものとする．


In [30]:
%%sql

SELECT 
    customer_id, 
    amount,
    RANK() OVER (ORDER BY amount DESC) AS ranking
FROM
    receipt
LIMIT 10;

 * sqlite:///../data/data-analytics-lecture.db
Done.


customer_id,amount,ranking
CS011415000006,10925,1
ZZ000000000000,6800,2
CS028605000002,5780,3
CS015515000034,5480,4
ZZ000000000000,5480,4
ZZ000000000000,5480,4
ZZ000000000000,5440,7
CS021515000089,5440,7
CS039415000073,5280,9
CS032414000072,5280,9


### Knock 34: Window関数2
レシート明細テーブル（`receipt`）の各レコードに対して1件あたりの売上金額（`amount`）が高い順にランク（順位）を付与し，先頭10件を抽出せよ．その際，項目として顧客ID（`customer_id`），売上金額（`amount`），付与したランクを表示させること．なお，売上金額が等しい場合は抜け番にせず，異なるランクを付与するものとする．

In [31]:
%%sql

SELECT 
    customer_id, 
    amount,
    ROW_NUMBER() OVER (ORDER BY amount DESC) AS ranking
FROM
    receipt
LIMIT 10;

 * sqlite:///../data/data-analytics-lecture.db
Done.


customer_id,amount,ranking
CS011415000006,10925,1
ZZ000000000000,6800,2
CS028605000002,5780,3
CS015515000034,5480,4
ZZ000000000000,5480,5
ZZ000000000000,5480,6
ZZ000000000000,5440,7
CS021515000089,5440,8
CS039415000073,5280,9
CS032414000072,5280,10


### Knock 35: Window関数3
レシート明細テーブル（`receipt`）から顧客ID（`customer_id`）が`CS040214000008`のレコードのみを取り出し，購買日（`sales_ymd`）でソートした上で以下の項目を表示せよ：
* 購買日（`sales_ymd`）
* その日の売上総額（`amount`）
* 全期間を通した当該顧客の売上総額（`total_amount`）

なお，表示件数は先頭の10件のみでよい．

In [32]:
%%sql

SELECT 
    DISTINCT sales_ymd,
    SUM(amount) OVER (PARTITION BY sales_ymd) AS amount,
    SUM(amount) OVER () AS total_amount
FROM
    receipt
WHERE
    customer_id = "CS040214000008"
ORDER BY
    sales_ymd;

 * sqlite:///../data/data-analytics-lecture.db
Done.


sales_ymd,amount,total_amount
20170119,1298,13523
20170123,416,13523
20170606,188,13523
20170612,976,13523
20170622,235,13523
20170713,1300,13523
20170824,168,13523
20171105,396,13523
20171127,1568,13523
20180103,258,13523


### Knock 36: Window関数4
レシート明細テーブル（`receipt`）から顧客ID（`customer_id`）が`CS040214000008`のレコードのみを取り出し，購買日（`sales_ymd`）でソートした上で以下の項目を表示せよ：
* 購買日（`sales_ymd`）
* その日の売上総額（`amount`）
* その日までの売上の累積値（`cumulative_total_amount`）

なお，表示件数は先頭の10件のみでよい．

In [33]:
%%sql

SELECT 
    DISTINCT sales_ymd,
    SUM(amount) OVER (PARTITION BY sales_ymd) AS amount,    
    SUM(amount) OVER (ORDER BY sales_ymd) AS cumulative_total_amount    
FROM
    receipt
WHERE
    customer_id = "CS040214000008"
ORDER BY
    sales_ymd;

 * sqlite:///../data/data-analytics-lecture.db
Done.


sales_ymd,amount,cumulative_total_amount
20170119,1298,1298
20170123,416,1714
20170606,188,1902
20170612,976,2878
20170622,235,3113
20170713,1300,4413
20170824,168,4581
20171105,396,4977
20171127,1568,6545
20180103,258,6803


### Knock 37: 副問い合わせ1
レシート明細テーブル（`receipt`）を利用して顧客ID（`customer_id`）ごとの売上総額を計算し，その平均（`avg_total_amount`）を求めよ．ただし，顧客IDが"Z"から始まるのものは非会員を表すため，除外して計算すること．

In [34]:
%%sql

SELECT
    AVG(total_amount) AS avg_total_amount
FROM (
    SELECT
        customer_id,
        SUM(amount) AS total_amount
    FROM
        receipt
    WHERE
        customer_id NOT LIKE "Z%"
    GROUP BY
        customer_id
);

 * sqlite:///../data/data-analytics-lecture.db
Done.


avg_total_amount
2547.742234529256


### Knock 38: 副問い合わせ2
レシート明細テーブル（`receipt`）を利用して全商品の中で売上総額が上位3位の商品コード（`product_cd`）を求め，各店舗（`store_cd`）における3つの商品の売上総額を求めよ．なお，結果は`store_cd`列でソートして表示すること．なお，このノックでは`WITH句`を用いよ．表示件数は30件でよい．

In [35]:
%%sql

WITH top_product AS (
    SELECT
        product_cd
    FROM
        receipt
    GROUP BY
        product_cd
    ORDER BY
        SUM(amount) DESC
    LIMIT 3   
)
SELECT
    store_cd,
    product_cd,
    SUM(amount) AS total_amount
FROM
    receipt
WHERE
    product_cd IN top_product
GROUP BY
    store_cd, product_cd
ORDER BY
    store_cd
LIMIT 30;

 * sqlite:///../data/data-analytics-lecture.db
Done.


store_cd,product_cd,total_amount
S12007,P060303001,12432
S12007,P071401001,6600
S12007,P071401002,23100
S12013,P060303001,18944
S12013,P071401001,74800
S12013,P071401002,11000
S12014,P060303001,11544
S12014,P071401001,42900
S12014,P071401002,12100
S12029,P060303001,16132


### Knock 39: 副問い合わせ3（難）
レシート明細テーブル（`df_receipt`）に対し，店舗コード（`store_cd`）ごとに商品コードごと（`product_cd`）の売上金額（`amount`）総額を計算し，各店舗で売上総額が最も大きかった商品コードとその売上総額を表示せよ．

※ヒント：Window関数の`OVER`句の中で`PARTION BY`句を使う

In [36]:
%%sql

SELECT
    store_cd,
    product_cd,
    total_amount
FROM (
    SELECT
        store_cd,
        product_cd,
        SUM(amount) AS total_amount,
        RANK() OVER(PARTITION BY store_cd ORDER BY SUM(amount) DESC) AS rank
    FROM
        receipt
    GROUP BY
        customer_id, product_cd
)
WHERE
    rank = 1
LIMIT 10

 * sqlite:///../data/data-analytics-lecture.db
Done.


store_cd,product_cd,total_amount
S12007,P071401006,102000
S12013,P080901002,40850
S12014,P071401022,134400
S12029,P071401012,151800
S12030,P040301005,14304
S13001,P040503001,111154
S13002,P071401001,629200
S13003,P071401016,127600
S13004,P071401007,123600
S13005,P040101001,32076


### Knock 40: 副問い合わせ4（難）
レシート明細テーブル（`receipt`）に対して商品コードごと（`product_cd`）の売上金額（`amount`）総額を計算し，売上総額のパーセンタイル値が25%から75%の商品コードを表示せよ．なお，表示する件数は10でよい．

※ヒント: Window関数の`PERCENT_RANK`を使う

In [37]:
%%sql

SELECT
    *
FROM (
    SELECT 
        *,
        PERCENT_RANK() OVER (ORDER BY total_amount ASC) AS percent_rank
    FROM (
        SELECT
            product_cd,
            SUM(amount) AS total_amount
        FROM
            receipt
        GROUP BY
            product_cd
    )
)
WHERE
    percent_rank BETWEEN 0.25 AND 0.75
LIMIT 10

 * sqlite:///../data/data-analytics-lecture.db
Done.


product_cd,total_amount,percent_rank
P050202031,438,0.2514702122219381
P070101129,438,0.2514702122219381
P070505060,438,0.2514702122219381
P070803062,438,0.2514702122219381
P070903039,438,0.2514702122219381
P090102081,438,0.2514702122219381
P090401036,438,0.2514702122219381
P050201007,440,0.2523651240092048
P070701023,440,0.2523651240092048
P070702012,440,0.2523651240092048
